In [4]:
import argparse
from functools import partial
from importlib import reload
from ipywidgets import Text, Label, Dropdown, Tab, HTML, HBox, VBox, Button, Layout
from pathlib import Path

import dataset.ndarray
import interface
reload(interface)
import scripts
reload(scripts)

##############################################################################
################################ Interface ###################################
##############################################################################

interface = interface.Interface()
interface.ensure_state("environment", value="local")

if interface.environment == "local":
    interface.ensure_state("drive_mount", value="/mnt/chromeos/MyFiles")
elif interface.environment == "colab":
    interface.ensure_state("drive_mount", value="/content/drive")

#################################### Setup ###################################

input__drive_folder = Text(description="Drive folder:")
interface.identity("drive_folder", input__drive_folder, value="")

@interface.property
def base_path(drive_mount, drive_folder, environment):
    if environment == "local":
        return Path(drive_mount) / drive_folder
    elif environment == "colab":
        return Path(drive_mount) / "MyDrive" / drive_folder


dropdown__case = Dropdown(description="Select case:")


@interface.inject(dropdown__case)
def options(base_path):
    return dataset.path_explorer.discover(base_path)


interface.identity("case", dropdown__case)


@interface.property
def case_path(drive_mount, drive_folder, case):
    if case:
        return Path(drive_mount) / drive_folder / case
    

label__isdicom = Label()
@interface.inject(label__isdicom)
def value(case, case_path):
    if case and dataset.path_explorer.is_dicom(case_path):
        return f"Case {case} is a dicomdir."
    else:
        return f"Case {case} is NOT a dicomdir."

label__original = Label()
@interface.inject(label__original)
def value(case, case_path):
    if case and dataset.path_explorer.is_original(case_path):
        return f"Case {case} has the original scans converted to nifti."
    else:
        return f"Case {case} does not have the original scans in nifti format."

label__registered = Label()
@interface.inject(label__registered)
def value(case, case_path):
    if case and dataset.path_explorer.is_registered(case_path):
        return f"Case {case} is registered."
    else:
        return f"Case {case} does not have the registered scans."
    
interface.new_tab(
    "Setup",
    HBox([
        VBox([
            HTML("<h3>Set these variables</h3>"),
            input__drive_folder,
            dropdown__case,
        ]),
        VBox([
            HTML("<h3>About the selected case:</h3>"),
            label__isdicom,
            label__original,
            label__registered,
        ]),
    ])
)

############################### Preprocessing ################################

button__convert = Button(layout=Layout(width='auto'))

@interface.inject(button__convert)
def description(case, case_path):
    if case:
        if dataset.path_explorer.is_dicom(case_path):
            if dataset.path_explorer.is_original(case_path):
                return f"Convert only {str(case)} (overwrite!)"
            else:
                return f"Convert only {str(case)}"
        else:
            return f"Can't convert {case} because it is not a dicomdir."
    else:
        return "No case selected!"

@interface.inject(button__convert)
def disabled(case, case_path):
    if case:
        if dataset.path_explorer.is_dicom(case_path):
            return False
    print("DIS")
    return True

@interface.inject(button__convert)
def button_style(case, case_path):
    if case and dataset.path_explorer.is_dicom(case_path) and dataset.path_explorer.is_original(case_path):
        return "warning"
    return "info"

@interface.on_click(button__convert, "case_path", overwrite=True)
def callback(*, case_path, overwrite):
    opts = argparse.Namespace(
        sources=case_path,
        outputs=case_path,
        overwrite=overwrite
    )
    scripts.dicom2nifti.main(opts)

button__niftyreg = Button(layout=Layout(width='auto'))

@interface.inject(button__niftyreg)
def description(case, case_path):
    if case:
        if dataset.path_explorer.is_original(case_path):
            if dataset.path_explorer.is_registered(case_path):
                return f"Register {str(case)} with NiftyReg (overwrite!)"
            else:
                return f"Register {str(case)} with NiftyReg"
        else:
            return f"Can't register {case} because there are no originals."
    else:
        return "No case selected!"

@interface.inject(button__niftyreg)
def disabled(case, case_path):
    if case:
        if dataset.path_explorer.is_original(case_path):
            return False
    return True

@interface.inject(button__niftyreg)
def button_style(case, case_path):
    if case and dataset.path_explorer.is_original(case_path) and dataset.path_explorer.is_registered(case_path):
        return "warning"
    return "info"

@interface.on_click(button__niftyreg, "case_path", overwrite=True)
def callback(*, case_path, overwrite):
    opts = argparse.Namespace(
        sources=case_path,
        outputs=case_path,
        overwrite=overwrite,
        niftybin="/usr/local/bin"
    )
    scripts.niftyreg.main(opts)

button__pyelastix = Button(layout=Layout(width='auto'))

@interface.inject(button__pyelastix)
def description(case, case_path):
    if case:
        if dataset.path_explorer.is_original(case_path):
            if dataset.path_explorer.is_registered(case_path):
                return f"Register {str(case)} with PyElastix (overwrite!)"
            else:
                return f"Register {str(case)} with PyElastix"
        else:
            return f"Can't register {case} because there are no originals."
    else:
        return "No case selected!"

@interface.inject(button__pyelastix)
def disabled(case, case_path):
    if case:
        if dataset.path_explorer.is_original(case_path):
            return False
    return True

@interface.inject(button__pyelastix)
def button_style(case, case_path):
    if case and dataset.path_explorer.is_original(case_path) and dataset.path_explorer.is_registered(case_path):
        return "warning"
    return "info"

@interface.on_click(button__pyelastix, "case_path", overwrite=True)
def callback(*, case_path, overwrite):
    opts = argparse.Namespace(
        sources=case_path,
        outputs=case_path,
        overwrite=overwrite,
    )
    scripts.pyelastix.main(opts)
    
interface.new_tab(
    "Preprocessing",
    HBox([
        VBox([
            HTML("<h3>Single case operations</h3>"),
            button__convert,
            button__niftyreg,
            button__pyelastix,
            HTML("<h3>Operations on the whole dataset</h3>"),
            
        ])
    ])
)

################################### Tests ####################################



input__drive_mount = Text(description="Drive mount:")
interface.identity("drive_mount", input__drive_mount, value="/mnt/chromeos/MyFiles")


mount_from_colab = Button(button_style='info', description="Mount from colab", layout=Layout(width='auto'))

@interface.on_click(mount_from_colab, "drive_mount")
def callback(*, base_path):
    !pytest test_package/test_ndarray.py

test_ndarray = Button(button_style='info', description="Test ndarray", layout=Layout(width='auto'))

@interface.on_click(test_ndarray, "base_path")
def callback(*, base_path):
    !pytest test_package/test_ndarray.py

test_path_explorer = Button(button_style='info', description="Test path explorer", layout=Layout(width='auto'))

@interface.on_click(test_path_explorer, "base_path")
def callback(*, base_path):
    !pytest test_package/test_path_explorer.py

test_preprocessing = Button(button_style='info', description="Test preprocessing", layout=Layout(width='auto'))

@interface.on_click(test_preprocessing, "base_path")
def callback(*, base_path):
    !pytest test_package/test_preprocessing.py

test_slicing = Button(button_style='info', description="test_slicing", layout=Layout(width='auto'))

@interface.on_click(test_slicing, "base_path")
def callback(*, base_path):
    !pytest test_package/test_slicing.py
    
interface.new_tab(
    "Developer's tests",
    HBox([
        VBox([
            HTML("<h3>This is not the tab you are looking for</h3>"),
            input__drive_mount,
            test_ndarray,
            test_path_explorer,
            test_preprocessing,
            test_slicing
        ])
    ])
)

##############################################################################

display(interface)
interface.drive_folder = "Downloads"

Interface(children=(HBox(children=(VBox(children=(HTML(value='<h3>Set these variables</h3>'), Text(value='', d…

# Other stuff

In [5]:
Button(button_style='info', description="Test ndarray", disabled=False, layout=Layout(width='auto'))

Button(button_style='info', description='Test ndarray', layout=Layout(width='auto'), style=ButtonStyle())